In [26]:
!pip3 install transformers datasets torch
!pip3 install --upgrade transformers
!pip3 install -q -U peft
!pip3 install -q -U trl
!pip3 install 'accelerate>=0.26.0'
!pip3 install bitsandbytes
!pip3 install -q numba


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [27]:
import os
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import BitsAndBytesConfig
import transformers
import torch
from datasets import load_dataset
from trl import SFTTrainer , SFTConfig
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig, GemmaTokenizer, TrainingArguments

In [28]:

file_path = '/teamspace/studios/this_studio/fuktdata.json'
dst = load_dataset("json",data_files=file_path)


In [29]:
dst

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 241
    })
})

In [30]:
trts = dst['train'].train_test_split(test_size=0.2)
tst = trts['test']
trn = trts['train']
print(trn)
print()
print(tst)

Dataset({
    features: ['input', 'output'],
    num_rows: 192
})

Dataset({
    features: ['input', 'output'],
    num_rows: 49
})


In [31]:
print(trts)
print(tst)
print(trn)

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 192
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 49
    })
})
Dataset({
    features: ['input', 'output'],
    num_rows: 49
})
Dataset({
    features: ['input', 'output'],
    num_rows: 192
})


In [32]:
from transformers import AutoTokenizer, AutoModelForCausalLM , AutoModelForSeq2SeqLM

In [33]:

load_dotenv()

hf_tok = os.getenv('HF_TOK')

modid = "google/gemma-2-2b-it"

In [34]:
tkizer = AutoTokenizer.from_pretrained(modid, token=hf_tok)

In [35]:
tkizer.padding_side = "right"

In [36]:
def data_tkizer(ex):
    iptxt = ex['input']
    return tkizer(iptxt, truncation=True, padding="max_length",max_length=512)
    

In [37]:
trn = trn.map(data_tkizer, batched=True)
tst = tst.map(data_tkizer, batched=True)

In [38]:
quant = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.bfloat16
)

In [39]:
mod = AutoModelForCausalLM.from_pretrained(modid,quantization_config = quant, device_map={"":0},token = hf_tok)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [40]:
txt = "Teach me deep learning i'm a beginner"
device = "cuda:0"
ip = tkizer(txt, return_tensors="pt").to(device)

op=mod.generate(**ip,max_new_tokens=512)


In [41]:
print(tkizer.decode(op[0],skip_special_tokens=True))

Teach me deep learning i'm a beginner.

## Deep Learning for Beginners: A Gentle Introduction

Welcome to the world of deep learning! It's exciting, challenging, and rewarding. This guide will help you understand the basics and get started.

**What is Deep Learning?**

Deep learning is a subfield of machine learning that uses artificial neural networks (ANNs) to learn from data. These networks are inspired by the human brain and can learn complex patterns and relationships.

**Key Concepts:**

* **Artificial Neural Networks (ANNs):** The building blocks of deep learning. They consist of interconnected nodes (neurons) organized in layers.
* **Layers:**  Each layer performs a specific function on the input data.
* **Activation Functions:**  Non-linear functions that introduce complexity and allow the network to learn non-linear relationships.
* **Training:** The process of adjusting the network's weights and biases to minimize the difference between its predictions and the actual values.

In [42]:
lora_config = LoraConfig(
    r = 8,
    target_modules = ["q_proj", "o_proj", "k_proj", "v_proj",
                      "gate_proj", "up_proj", "down_proj"],
    task_type = "CAUSAL_LM",
)

In [43]:
print(tst['input'])
tst['output']

['I need to improve my analytical skills.', 'I plan to learn Photoshop in two weeks. I have intermediate skills.', 'I wish to learn Russian.', 'I want to learn how to play basketball in one month.', 'I wish to study geology over the next semester.', 'I wish to learn advanced calculus in a month.', 'I aim to master advanced physics in 6 months.', 'I wish to learn blockchain development in two months.', 'I plan to learn data structures and algorithms.', 'I plan to become fluent in French.', 'I wish to learn first aid in two weeks.', 'I want to learn how to make mobile apps in three months.', 'I want to improve my painting skills in three months. I am at a basic level.', 'I aim to improve my algebra skills in one month.', 'I plan to learn how to play chess.', 'I want to learn business analytics in two months.', 'I aim to master the art of negotiation.', 'I wish to learn Portuguese.', 'I wish to study chemistry over the next semester.', 'I plan to learn AWS cloud services in three months.'

[{'goal': 'Improve analytical skills',
  'knowledge_level': 'Intermediate',
  'response': None,
  'time_frame': 'Not specified'},
 {'goal': 'Learn Photoshop',
  'knowledge_level': 'Intermediate',
  'response': None,
  'time_frame': 'Two weeks'},
 {'goal': 'Learn Russian',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'Not specified'},
 {'goal': 'Learn basketball',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'One month'},
 {'goal': 'Learn geology',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'Four months'},
 {'goal': 'Learn advanced calculus',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'One month'},
 {'goal': 'Learn advanced physics',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'Six months'},
 {'goal': 'Learn blockchain development',
  'knowledge_level': 'Beginner',
  'response': None,
  'time_frame': 'Two months'},
 {'goal': 'Learn data structures and algorithms',

In [44]:
def format_data(ex):
    txxt = f"input: {ex['input'][0]}\noutput: {ex['output'][0]}"
    return [txxt]
# format_data(trn)

In [45]:
trainer = SFTTrainer(
    model=mod,
    train_dataset=trn,
    eval_dataset=tst,
    tokenizer=tkizer,

    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        max_steps=100,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        weight_decay=0.01,
        output_dir = "/teamspace/studios/this_studio/results",
        logging_dir = "/teamspace/studios/this_studio/logs",
        optim="paged_adamw_8bit"
    ),
    peft_config=lora_config,
    # formatting_func=formatting_func,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


In [46]:
trainer.train()

Step,Training Loss
1,3.884000
2,3.803500
3,3.979200
4,3.911800
5,3.646900
6,3.356600
7,2.606000
8,2.313900
9,1.805100
10,2.081000


TrainOutput(global_step=100, training_loss=1.343193746805191, metrics={'train_runtime': 298.2772, 'train_samples_per_second': 2.682, 'train_steps_per_second': 0.335, 'total_flos': 5000973307084800.0, 'train_loss': 1.343193746805191, 'epoch': 4.166666666666667})

In [48]:
traineval = trainer.evaluate()
print(traineval)

{'eval_loss': 1.358211636543274, 'eval_runtime': 9.2005, 'eval_samples_per_second': 5.326, 'eval_steps_per_second': 0.761, 'epoch': 4.166666666666667}


In [56]:
mod.save_pretrained("./fine-tuned-gemma")
tkizer.save_pretrained("./fine-tuned-gemma")

('./fine-tuned-gemma/tokenizer_config.json',
 './fine-tuned-gemma/special_tokens_map.json',
 './fine-tuned-gemma/tokenizer.json')

In [59]:
ftmod = AutoModelForCausalLM.from_pretrained("./fine-tuned-gemma")
tk = AutoTokenizer.from_pretrained("./fine-tuned-gemma",quantization_config = quant, device_map={"":0})

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Some weights of the model checkpoint at ./fine-tuned-gemma were not used when initializing Gemma2ForCausalLM: ['model.layers.0.mlp.down_proj.base_layer.weight', 'model.layers.0.mlp.down_proj.base_layer.weight.absmax', 'model.layers.0.mlp.down_proj.base_layer.weight.quant_map', 'model.layers.0.mlp.down_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.mlp.down_proj.lora_A.default.weight', 'model.layers.0.mlp.down_proj.lora_B.default.weight', 'model.layers.0.mlp.gate_proj.base_layer.weight', 'model.layers.0.mlp.gate_proj.base_layer.weight.absmax', 'model.layers.0.mlp.gate_proj.base_layer.weight.quant_map', 'model.layers.0.mlp.gate_proj.base_layer.weight.quant_state.bitsandbytes__nf4', 'model.layers.0.mlp.gate_proj.lora_A.default.weigh

In [58]:
txt = "I want to learn Classical physics in a week iam a beginner"
device = "cuda:0"
ip = tk(txt, return_tensors="pt").to(device)

op=ftmod.generate(**ip,max_new_tokens=512)

print(tk.decode(op[0],skip_special_tokens=True))

I want to learn Classical physics in a week iam a beginner sula kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasarigan kasari